<a href="https://colab.research.google.com/github/feknall/machinelearning-playground/blob/main/DeepLearning_Assignment_2_Adam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Installation

In [ ]:
%load_ext tensorboard

In [ ]:
! apt install ttf-mscorefonts-installer

In [ ]:
! fc-cache -f

In [ ]:
! fc-match Arial

### Loading MNIST

In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

(x_train_both, y_train_both), (x_test, y_test) = mnist.load_data()

print(x_train_both[x_train_both > 0])

x_train_both = x_train_both.reshape((x_train_both.shape[0], -1)).astype('float32') / 255
x_test = x_test.reshape((x_test.shape[0], -1)).astype('float32') / 255

scaler = StandardScaler()
x_train_both = scaler.fit_transform(x_train_both)
x_test = scaler.transform(x_test)

print(x_train_both[x_train_both > 0])

x_train, x_validation, y_train, y_validation = train_test_split(x_train_both, y_train_both, test_size=0.2, shuffle=True)

In [ ]:
from sklearn.preprocessing import OneHotEncoder

one_hot_encoder = OneHotEncoder()
one_hot_encoder.fit(y_train.reshape(-1, 1))
y_train_one_hot_encoded = one_hot_encoder.transform(y_train.reshape(-1, 1)).toarray()
y_validation_hot_encoded = one_hot_encoder.transform(y_validation.reshape(-1, 1)).toarray()

In [ ]:
print(f"y_train: \n{y_train.reshape(-1, 1)[:5]}")
print(f"y_train_one_hot_encoded: \n{y_train_one_hot_encoded[:5]}")
print(f"y_validation_hot_encoded: \n{y_validation_hot_encoded[:3]}")

In [ ]:
print(x_train[0][:100])

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### SGD Classifier

In [ ]:
! pip install tensorflow --upgrade

In [ ]:
! pip install keras-tuner --upgrade

In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape((x_train.shape[0], -1)).astype('float32') / 255
x_test = x_test.reshape((x_test.shape[0], -1)).astype('float32') / 255

scaler = StandardScaler()
scaler.fit(x_train)
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

In [ ]:
from sklearn.preprocessing import OneHotEncoder

one_hot_encoder = OneHotEncoder()
one_hot_encoder.fit(y_train.reshape(-1, 1))
y_train_one_hot_encoded = one_hot_encoder.transform(y_train.reshape(-1, 1)).toarray()

### Keras Adam

In [ ]:
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator
import tensorflow as tf

def plot_tensorflow_log(path):

    # Loading too much data is slow...
    tf_size_guidance = {
        # 'compressedHistograms': 100,
        # 'images': 0,
        # 'scalars': 10000,
        # 'histograms': 1000
        'tensors': 200
    }

    event_acc = EventAccumulator(path, tf_size_guidance)
    event_acc.Reload()

    print(event_acc.Tags())

    training_accuracies = event_acc.Tensors('epoch_accuracy')
    tensor_np = tf.make_ndarray(training_accuracies[-1].tensor_proto)

    print(tensor_np)

    y = list()
    x = list()
    for i in range(len(training_accuracies)):
        y.append(tf.make_ndarray(training_accuracies[i].tensor_proto))
        x.append(i)

    return x, y

In [ ]:
experiment_train_size = 20000
X = x_train[:experiment_train_size]
y = y_train_one_hot_encoded[:experiment_train_size]

## Learning Rate

In [ ]:
from tensorflow import keras
from tensorflow.keras.layers import Dense

def build_model(hp):
  learning_rate = hp.Choice("learning_rate", [0.1, 0.01, 0.001, 0.0001])

  model = keras.Sequential()
  model.add(Dense(10, kernel_initializer='uniform', input_shape=(784,), activation='softmax'))

  opt = keras.optimizers.Adam(learning_rate=learning_rate)
  model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=["accuracy"])
  return model

In [ ]:
import keras_tuner as kt
from tensorflow import keras

tuner = kt.RandomSearch(
    hypermodel=build_model,
    objective="val_accuracy",
    max_trials=1000,
    executions_per_trial=1,
    overwrite=True,
    directory="/content/drive/MyDrive/deep-learning/adam/",
    project_name="learning_rate_project"
)

tuner.search(X, 
             y, 
             epochs=20, 
             validation_split=0.2, 
             callbacks=[tf.keras.callbacks.TensorBoard("/content/drive/MyDrive/deep-learning/adam/learning_rate_project/callback")],
          
)

In [ ]:
! rm -rf /content/drive/MyDrive/deep-learning/adam/learning_rate_project

In [ ]:
%tensorboard --logdir /content/drive/MyDrive/deep-learning/adam/learning_rate_project/callback

In [ ]:
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt


log_file = "/content/drive/MyDrive/deep-learning/adam/learning_rate_project/callback/0000/execution0/train/events.out.tfevents.1649280351.6ba8d469f996.71.19.v2"
learing_rate_01_train_x, learing_rate_01_train_y = plot_tensorflow_log(log_file)
log_file = "/content/drive/MyDrive/deep-learning/adam/learning_rate_project/callback/0000/execution0/validation/events.out.tfevents.1649280353.6ba8d469f996.71.20.v2"
learing_rate_01_val_x, learing_rate_01_val_y = plot_tensorflow_log(log_file)

log_file = "/content/drive/MyDrive/deep-learning/adam/learning_rate_project/callback/0001/execution0/train/events.out.tfevents.1649280393.6ba8d469f996.71.22.v2"
learing_rate_00001_train_x, learing_rate_00001_train_y = plot_tensorflow_log(log_file)
log_file = "/content/drive/MyDrive/deep-learning/adam/learning_rate_project/callback/0001/execution0/validation/events.out.tfevents.1649280395.6ba8d469f996.71.23.v2"
learing_rate_00001_val_x, learing_rate_00001_val_y = plot_tensorflow_log(log_file)

log_file = "/content/drive/MyDrive/deep-learning/adam/learning_rate_project/callback/0002/execution0/train/events.out.tfevents.1649280431.6ba8d469f996.71.25.v2"
learing_rate_001_train_x, learing_rate_001_train_y = plot_tensorflow_log(log_file)
log_file = "/content/drive/MyDrive/deep-learning/adam/learning_rate_project/callback/0002/execution0/validation/events.out.tfevents.1649280433.6ba8d469f996.71.26.v2"
learing_rate_001_val_x, learing_rate_001_val_y = plot_tensorflow_log(log_file)

log_file = "/content/drive/MyDrive/deep-learning/adam/learning_rate_project/callback/0003/execution0/train/events.out.tfevents.1649280468.6ba8d469f996.71.28.v2"
learing_rate_0001_train_x, learing_rate_0001_train_y = plot_tensorflow_log(log_file)
log_file = "/content/drive/MyDrive/deep-learning/adam/learning_rate_project/callback/0003/execution0/validation/events.out.tfevents.1649280470.6ba8d469f996.71.29.v2"
learing_rate_0001_val_x, learing_rate_0001_val_y = plot_tensorflow_log(log_file)

plt.plot(learing_rate_01_train_x, learing_rate_01_train_y, label='0.01 learning rate - train')
plt.plot(learing_rate_00001_train_x, learing_rate_00001_train_y, label='0.0001 learning rate - train')
plt.plot(learing_rate_001_train_x, learing_rate_001_train_y, label='0.01 learning rate - train')
plt.plot(learing_rate_0001_train_x, learing_rate_0001_train_y, label='0.001 learning rate - train')

plt.plot(learing_rate_01_val_x, learing_rate_01_val_y, '--', label='0.01 learning rate - train')
plt.plot(learing_rate_00001_val_x, learing_rate_00001_val_y, '--', label='0.0001 learning rate - train')
plt.plot(learing_rate_001_val_x, learing_rate_001_val_y, '--', label='0.01 learning rate - train')
plt.plot(learing_rate_0001_val_x, learing_rate_0001_val_y, '--', label='0.001 learning rate - train')

plt.xlabel("Steps")
plt.ylabel("Accuracy")
plt.title("Training Progress - Adam")
plt.legend(loc='down right', frameon=True)
plt.show()

## Initializer

In [ ]:
from tensorflow import keras
from tensorflow.keras.layers import Dense
from tensorflow.keras import initializers

def build_model(hp):
  initializer = hp.Choice("initializer", ['random_normal',
                                          'random_uniform', 
                                          'zeros', 
                                          'ones'])
  
  model = keras.Sequential()
  model.add(Dense(10, kernel_initializer=initializer, input_shape=(784,), activation='softmax'))

  opt = keras.optimizers.Adam(learning_rate=0.001)
  model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=["accuracy"])
  return model

In [ ]:
import keras_tuner as kt
from tensorflow import keras

tuner = kt.RandomSearch(
    hypermodel=build_model,
    objective="val_accuracy",
    max_trials=1000,
    executions_per_trial=1,
    overwrite=True,
    directory="/content/drive/MyDrive/deep-learning/",
    project_name="initializer_project"
)

tuner.search(X, 
             y, 
             epochs=20, 
             validation_split=0.2, 
             callbacks=[tf.keras.callbacks.TensorBoard("/content/drive/MyDrive/deep-learning/initializer_project/callback")],
          
)

In [ ]:
! rm -rf /content/drive/MyDrive/deep-learning/initializer_project/callback

In [ ]:
%tensorboard --logdir /content/drive/MyDrive/deep-learning/initializer_project/callback

In [ ]:
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt



log_file = "/content/drive/MyDrive/deep-learning/initializer_project/callback/0002/execution0/train/events.out.tfevents.1648509262.baed5f330a88.60.63.v2"
x, y = plot_tensorflow_log(log_file)
plt.plot(x, y, label='ones - train')

log_file = "/content/drive/MyDrive/deep-learning/initializer_project/callback/0001/execution0/train/events.out.tfevents.1648509237.baed5f330a88.60.60.v2"
x, y = plot_tensorflow_log(log_file)
plt.plot(x, y, label='zeros - train')

log_file = "/content/drive/MyDrive/deep-learning/initializer_project/callback/0003/execution0/train/events.out.tfevents.1648509286.baed5f330a88.60.66.v2"
x, y = plot_tensorflow_log(log_file)
plt.plot(x, y, label='random normal - train')

log_file = "/content/drive/MyDrive/deep-learning/initializer_project/callback/0000/execution0/train/events.out.tfevents.1648509209.baed5f330a88.60.57.v2"
x, y = plot_tensorflow_log(log_file)
plt.plot(x, y, label='random uniform - train')


log_file = "/content/drive/MyDrive/deep-learning/initializer_project/callback/0002/execution0/validation/events.out.tfevents.1648509263.baed5f330a88.60.64.v2"
x, y = plot_tensorflow_log(log_file)
plt.plot(x, y, '--', label='ones - validation')

log_file = "/content/drive/MyDrive/deep-learning/initializer_project/callback/0001/execution0/validation/events.out.tfevents.1648509238.baed5f330a88.60.61.v2"
x, y = plot_tensorflow_log(log_file)
plt.plot(x, y, '--', label='zeros - validation')

log_file = "/content/drive/MyDrive/deep-learning/initializer_project/callback/0003/execution0/validation/events.out.tfevents.1648509288.baed5f330a88.60.67.v2"
x, y = plot_tensorflow_log(log_file)
plt.plot(x, y, '--', label='random normal - validation')

log_file = "/content/drive/MyDrive/deep-learning/initializer_project/callback/0000/execution0/validation/events.out.tfevents.1648509213.baed5f330a88.60.58.v2"
x, y = plot_tensorflow_log(log_file)
plt.plot(x, y, '--', label='random uniform - validation')

plt.xlabel("Steps")
plt.ylabel("Accuracy")
plt.title("Training Progress - Adam")
plt.legend(loc='down right', frameon=True)
plt.show()

## Batch Size

In [ ]:
class MyHyperModel(kt.HyperModel):
    def build(self, hp):  
      model = keras.Sequential()
      model.add(Dense(10, kernel_initializer='uniform', input_shape=(784,), activation='softmax'))

      opt = keras.optimizers.Adam(learning_rate=0.001)
      model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=["accuracy"])
      return model

    def fit(self, hp, model, *args, **kwargs):
        return model.fit(
            *args,
            batch_size=hp.Choice("batch_size", [8, 32, 128, 512, 2048]),
            **kwargs,
        )

In [ ]:
import keras_tuner as kt
from tensorflow import keras

tuner = kt.RandomSearch(
    hypermodel=MyHyperModel(),
    objective="val_accuracy",
    max_trials=1000,
    executions_per_trial=1,
    overwrite=True,
    directory="/content/drive/MyDrive/deep-learning/",
    project_name="batch_size_project"
)

tuner.search(X, 
             y, 
             epochs=20, 
             validation_split=0.2, 
             callbacks=[tf.keras.callbacks.TensorBoard("/content/drive/MyDrive/deep-learning/batch_size_project/callback")],
          
)

In [ ]:
! rm -rf /content/drive/MyDrive/deep-learning/batch_size_project/callback

In [ ]:
%tensorboard --logdir /content/drive/MyDrive/deep-learning/batch_size_project/callback

In [ ]:
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt


log_file = "/content/drive/MyDrive/deep-learning/batch_size_project/callback/0003/execution0/train/events.out.tfevents.1648510593.baed5f330a88.60.78.v2"
x, y = plot_tensorflow_log(log_file)
plt.plot(x, y, label='8 batch size - train')

log_file = "/content/drive/MyDrive/deep-learning/batch_size_project/callback/0004/execution0/train/events.out.tfevents.1648510678.baed5f330a88.60.81.v2"
x, y = plot_tensorflow_log(log_file)
plt.plot(x, y, label='32 batch size - train')

log_file = "/content/drive/MyDrive/deep-learning/batch_size_project/callback/0002/execution0/train/events.out.tfevents.1648510584.baed5f330a88.60.75.v2"
x, y = plot_tensorflow_log(log_file)
plt.plot(x, y, label='128 batch size - train')

log_file = "/content/drive/MyDrive/deep-learning/batch_size_project/callback/0005/execution0/train/events.out.tfevents.1648510703.baed5f330a88.60.84.v2"
x, y = plot_tensorflow_log(log_file)
plt.plot(x, y, label='512 batch size - train')

log_file = "/content/drive/MyDrive/deep-learning/batch_size_project/callback/0001/execution0/train/events.out.tfevents.1648510579.baed5f330a88.60.72.v2"
x, y = plot_tensorflow_log(log_file)
plt.plot(x, y, label='2048 batch size - train')



log_file = "/content/drive/MyDrive/deep-learning/batch_size_project/callback/0003/execution0/validation/events.out.tfevents.1648510597.baed5f330a88.60.79.v2"
x, y = plot_tensorflow_log(log_file)
plt.plot(x, y, '--', label='8 batch size - validation')

log_file = "/content/drive/MyDrive/deep-learning/batch_size_project/callback/0004/execution0/validation/events.out.tfevents.1648510680.baed5f330a88.60.82.v2"
x, y = plot_tensorflow_log(log_file)
plt.plot(x, y, '--', label='32 batch size - validation')

log_file = "/content/drive/MyDrive/deep-learning/batch_size_project/callback/0002/execution0/validation/events.out.tfevents.1648510585.baed5f330a88.60.76.v2"
x, y = plot_tensorflow_log(log_file)
plt.plot(x, y, '--', label='128 batch size - validation')

log_file = "/content/drive/MyDrive/deep-learning/batch_size_project/callback/0005/execution0/validation/events.out.tfevents.1648510703.baed5f330a88.60.85.v2"
x, y = plot_tensorflow_log(log_file)
plt.plot(x, y, '--', label='512 batch size - validation')

log_file = "/content/drive/MyDrive/deep-learning/batch_size_project/callback/0001/execution0/validation/events.out.tfevents.1648510579.baed5f330a88.60.73.v2"
x, y = plot_tensorflow_log(log_file)
plt.plot(x, y, '--', label='2048 batch size - validation')


plt.xlabel("Steps")
plt.ylabel("Accuracy")
plt.title("Training Progress - Adam")
plt.legend(loc='down right', frameon=True)
plt.show()

## Other

In [ ]:
predicted = model.predict(x_test)
print(predicted[0:10])

predicted_decode = np.argmax(predicted, axis=1)
print(predicted_decode)

accuracy_score(y_test, predicted_decode)

### Keras Adadelta

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

adadelta_model = keras.Sequential()
adadelta_model.add(layers.Dense(10, kernel_initializer='uniform', input_shape=(784,), activation='relu'))
adadelta_model.add(layers.Activation('sigmoid'))

opt = keras.optimizers.Adadelta(learning_rate=0.001)
adadelta_model.compile(loss='categorical_crossentropy', optimizer=opt, )

adadelta_model.fit(x_train, y_train_one_hot_encoded, epochs=10)

In [ ]:
predicted = adadelta_model.predict(x_test)
predicted[0:10]

predicted_decode = np.argmax(predicted, axis=1)

accuracy_score(y_test, predicted_decode)

### Keras SGD

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

adadelta_model = keras.Sequential()
adadelta_model.add(layers.Dense(10, kernel_initializer='uniform', input_shape=(784,), activation='relu'))
adadelta_model.add(layers.Activation('sigmoid'))

opt = keras.optimizers.SGD(learning_rate=0.001)
adadelta_model.compile(loss='categorical_crossentropy', optimizer=opt, )

adadelta_model.fit(x_train, y_train_one_hot_encoded, epochs=10, batch_size=32)

In [ ]:
predicted = adadelta_model.predict(x_test)
predicted[0:10]

predicted_decode = np.argmax(predicted, axis=1)

accuracy_score(y_test, predicted_decode)

### Keras RMSprop

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

adadelta_model = keras.Sequential()
adadelta_model.add(layers.Dense(10, kernel_initializer='uniform', input_shape=(784,), activation='relu'))
adadelta_model.add(layers.Activation('sigmoid'))

opt = keras.optimizers.RMSprop(learning_rate=0.001)
adadelta_model.compile(loss='categorical_crossentropy', optimizer=opt, )

adadelta_model.fit(x_train, y_train_one_hot_encoded, epochs=10, batch_size=32)

### L2 Regularization

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers

adadelta_model = keras.Sequential()
adadelta_model.add(layers.Dense(10, kernel_initializer='uniform', input_shape=(784,), activation='relu', kernel_regularizer=regularizers.l2(1e-4)))
adadelta_model.add(layers.Activation('sigmoid'))

opt = keras.optimizers.RMSprop(learning_rate=0.001)
adadelta_model.compile(loss='categorical_crossentropy', optimizer=opt, )

adadelta_model.fit(x_train, y_train_one_hot_encoded, epochs=100, batch_size=32)

### L1 Regularization

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers

adadelta_model = keras.Sequential()
adadelta_model.add(layers.Dense(10, kernel_initializer='uniform', input_shape=(784,), activation='relu', kernel_regularizer=regularizers.l1(1e-4)))
adadelta_model.add(layers.Activation('sigmoid'))

opt = keras.optimizers.RMSprop(learning_rate=0.001)
adadelta_model.compile(loss='categorical_crossentropy', optimizer=opt, )

adadelta_model.fit(x_train, y_train_one_hot_encoded, epochs=10, batch_size=32)

### Data Augmentation

In [ ]:
def random_flip_on_probability(image):
    return tf.image.random_flip_up_down(image)

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers

import random 
data_augmentation = tf.keras.Sequential([
  # layers.RandomFlip(mode='horizontal'),
  # layers.Lambda(random_flip_on_probability),
  layers.RandomRotation(0.05),
])

In [ ]:
x_train[0].shape

In [ ]:
# from tensorflow.python.ops.numpy_ops import np_config
# np_config.enable_numpy_behavior()

In [ ]:
image = tf.cast(tf.expand_dims(x_train[1].reshape(28, 28, 1), 0), tf.float32)
# image = tf.cast(tf.expand_dims(x_train[1].reshape(28, 28), 0), tf.float32)

In [ ]:
# fliped = tf.image.random_flip_up_down(image, seed=1)
# plt.imshow(fliped[0].reshape(28, 28))

In [ ]:

import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
for i in range(9):
  augmented_image = data_augmentation(image)
  ax = plt.subplot(3, 3, i + 1)
  plt.imshow(augmented_image[0].reshape(28, 28))
  plt.axis("off")

In [ ]:
# x_tensor = tf.cast(tf.expand_dims(x_train.reshape(-1, 28, 28, 1), 0), tf.float32)
x_tensor = tf.cast(x_train.reshape(-1, 28, 28, 1), tf.float32)
augmented_x = data_augmentation(x_tensor).numpy()

In [ ]:
augmented_x.shape

In [ ]:
x_train.shape

In [ ]:
augmented_x_flat = augmented_x.reshape((augmented_x.shape[0], -1))

In [ ]:
augmented_x_flat.shape

In [ ]:
plt.figure(figsize=(10, 10))
for i in range(9):
  plt.subplot(3, 3, i + 1)
  plt.imshow(augmented_x[i].reshape(28, 28))
  plt.axis("off")

In [ ]:
import numpy as np

x_and_augmented = np.concatenate((augmented_x_flat, x_train))
y_and_augmented = np.concatenate((y_train_one_hot_encoded, y_train_one_hot_encoded))

In [ ]:
x_and_augmented.shape

In [ ]:
y_and_augmented.shape

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

adadelta_model = keras.Sequential()
adadelta_model.add(layers.Dense(10, kernel_initializer='uniform', input_shape=(784,), activation='relu'))
adadelta_model.add(layers.Activation('sigmoid'))

opt = keras.optimizers.SGD(learning_rate=0.001)
adadelta_model.compile(loss='categorical_crossentropy', optimizer=opt, )

adadelta_model.fit(x_and_augmented, y_and_augmented, epochs=10, batch_size=32)

In [ ]:
from sklearn.metrics import accuracy_score

predicted = adadelta_model.predict(x_test)
predicted[0:10]

predicted_decode = np.argmax(predicted, axis=1)

accuracy_score(y_test, predicted_decode)

### Dropout CNN

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, ZeroPadding2D


def build_model(hp):
  dropout = hp.Choice("dropout", [0.1, 0.3, 0.5])

  model = Sequential()

  stride_size = 1
  kernel_size = 3
  pooling_size = 4

  model.add(Conv2D(16, (kernel_size, kernel_size), strides=(stride_size, stride_size), activation='relu', kernel_initializer='he_uniform', padding='same'))
  model.add(MaxPooling2D((2, 2)))
  model.add(Conv2D(32, (kernel_size, kernel_size), strides=(stride_size, stride_size), activation='relu', kernel_initializer='he_uniform', padding='same'))
  model.add(MaxPooling2D((2, 2)))
  model.add(Conv2D(64, (kernel_size, kernel_size), strides=(stride_size, stride_size), activation='relu', kernel_initializer='he_uniform', padding='same'))
  model.add(MaxPooling2D((2, 2)))

  model.add(Flatten())
  model.add(Dense(16, activation='relu', kernel_initializer='he_uniform'))
  model.add(Dense(16, activation='relu', kernel_initializer='he_uniform'))
  model.add(Dense(16, activation='relu', kernel_initializer='he_uniform'))

  model.add(Dropout(dropout))
  model.add(Dense(10, activation='softmax'))

  model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

  return model

In [ ]:
import keras_tuner as kt

tuner = kt.RandomSearch(
    hypermodel=build_model,
    objective="val_accuracy",
    max_trials=1000,
    executions_per_trial=1,
    overwrite=True,
    directory="/content/drive/MyDrive/deep-learning/dropout_dir",
    project_name="dropout"
)

In [ ]:
x_train = x_train.reshape(-1, 28, 28, 1)

In [ ]:
y_train_one_hot_encoded.shape

In [ ]:
experiment_train_size = 20000
X = x_train[:experiment_train_size]
y = y_train_one_hot_encoded[:experiment_train_size]

In [ ]:
tuner.search(X, 
             y, 
             epochs=20, 
             validation_split=0.2, 
             callbacks=[tf.keras.callbacks.TensorBoard("/content/drive/MyDrive/deep-learning/dropout_dir/callback")],
)

In [ ]:
tuner.results_summary(num_trials=10)

In [ ]:
! rm -rf /content/drive/MyDrive/ai-gras-1/pooling_size_dir

In [ ]:
%tensorboard --logdir /content/drive/MyDrive/ai-gras-1/dropout_dir/callback

In [ ]:
import tensorboard as tb

In [ ]:
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt


log_file = "/content/drive/MyDrive/deep-learning/dropout_dir/callback/19b43427bf22fc3f3d2c8cb8ffeddc24/execution0/train/events.out.tfevents.1645936062.422681e7c282.70.11.v2"
dropout_1_train_x, dropout_1_train_y = plot_tensorflow_log(log_file)
log_file = "/content/drive/MyDrive/deep-learning/dropout_dir/callback/19b43427bf22fc3f3d2c8cb8ffeddc24/execution0/validation/events.out.tfevents.1645936077.422681e7c282.70.12.v2"
dropout_1_validation_x, dropout_1_validation_y = plot_tensorflow_log(log_file)

log_file = "/content/drive/MyDrive/deep-learning/dropout_dir/callback/dcf09ddea4ff8d89cb4591aa64ea6227/execution0/train/events.out.tfevents.1645936695.422681e7c282.70.17.v2"
dropout_3_train_x, dropout_3_train_y = plot_tensorflow_log(log_file)
log_file = "/content/drive/MyDrive/deep-learning/dropout_dir/callback/dcf09ddea4ff8d89cb4591aa64ea6227/execution0/validation/events.out.tfevents.1645936710.422681e7c282.70.18.v2"
dropout_3_validation_x, dropout_3_validation_y = plot_tensorflow_log(log_file)

log_file = "/content/drive/MyDrive/deep-learning/dropout_dir/callback/726c0d19a58adad10d9cdb9c404ac37f/execution0/train/events.out.tfevents.1645936372.422681e7c282.70.14.v2"
dropout_5_train_x, dropout_5_train_y = plot_tensorflow_log(log_file)
log_file = "/content/drive/MyDrive/deep-learning/dropout_dir/callback/726c0d19a58adad10d9cdb9c404ac37f/execution0/validation/events.out.tfevents.1645936386.422681e7c282.70.15.v2"
dropout_5_validation_x, dropout_5_validation_y = plot_tensorflow_log(log_file)

plt.plot(dropout_1_train_x, dropout_1_train_y, label='0.1 dropout - train')
plt.plot(dropout_3_train_x, dropout_3_train_y, label='0.3 dropout - train')
plt.plot(dropout_5_train_x, dropout_5_train_y, label='0.5 dropout - train')

plt.plot(dropout_1_validation_x, dropout_1_validation_y, '--', label='0.1 dropout - validation')
plt.plot(dropout_3_validation_x, dropout_3_validation_y, '--', label='0.3 dropout - validation')
plt.plot(dropout_5_validation_x, dropout_5_validation_y, '--', label='0.5 dropout - validation')

plt.xlabel("Steps")
plt.ylabel("Accuracy")
plt.title("Training Progress")
plt.legend(loc='down right', frameon=True)
plt.show()